In [1]:
from google.colab import userdata
import os
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [2]:
!kaggle datasets download -d singhdatascientist/for-image-classification-of-cheetah-vs-hyena

 98% 151M/154M [00:05<00:00, 34.4MB/s]
100% 154M/154M [00:05<00:00, 28.1MB/s]


In [3]:
!unzip "/content/for-image-classification-of-cheetah-vs-hyena.zip"

Archive:  /content/for-image-classification-of-cheetah-vs-hyena.zip
  inflating: train/cheetah/cheetah_000_resized.jpg  
  inflating: train/cheetah/cheetah_001_resized.jpg  
  inflating: train/cheetah/cheetah_002_resized.jpg  
  inflating: train/cheetah/cheetah_003_resized.jpg  
  inflating: train/cheetah/cheetah_004_resized.jpg  
  inflating: train/cheetah/cheetah_005_resized.jpg  
  inflating: train/cheetah/cheetah_006_resized.jpg  
  inflating: train/cheetah/cheetah_007_resized.jpg  
  inflating: train/cheetah/cheetah_008_resized.jpg  
  inflating: train/cheetah/cheetah_009_resized.jpg  
  inflating: train/cheetah/cheetah_010_resized.jpg  
  inflating: train/cheetah/cheetah_011_resized.jpg  
  inflating: train/cheetah/cheetah_012_resized.jpg  
  inflating: train/cheetah/cheetah_013_resized.jpg  
  inflating: train/cheetah/cheetah_014_resized.jpg  
  inflating: train/cheetah/cheetah_015_resized.jpg  
  inflating: train/cheetah/cheetah_016_resized.jpg  
  inflating: train/cheetah/chee

In [4]:
import tensorflow as tf
train_dir = '/content/train'
test_dir = '/content/validation'

In [5]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224,224),
    batch_size=32,
    label_mode='int'
)

Found 1800 files belonging to 2 classes.


In [6]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224,224),
    batch_size=32,
    label_mode='int'
)

Found 200 files belonging to 2 classes.


In [7]:
num_classes= 2

In [8]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32
)

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_dataset, epochs=5, validation_data=test_dataset)
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy:', accuracy)

Found 1800 files belonging to 2 classes.
Found 200 files belonging to 2 classes.
Epoch 1/5
57/57 [==============================] - 12s 82ms/step - loss: 0.9023 - accuracy: 0.5644 - val_loss: 0.5915 - val_accuracy: 0.6500
Epoch 2/5
57/57 [==============================] - 4s 69ms/step - loss: 0.5116 - accuracy: 0.7572 - val_loss: 0.3537 - val_accuracy: 0.8450
Epoch 3/5
57/57 [==============================] - 5s 85ms/step - loss: 0.3752 - accuracy: 0.8411 - val_loss: 0.2807 - val_accuracy: 0.8950
Epoch 4/5
57/57 [==============================] - 5s 74ms/step - loss: 0.2911 - accuracy: 0.8828 - val_loss: 0.2735 - val_accuracy: 0.8850
Epoch 5/5
7/7 [==============================] - 0s 13ms/step - loss: 0.2426 - accuracy: 0.8950
Test accuracy: 0.8949999809265137


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        1

In [10]:
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications import ResNet50

In [11]:
base_model=ResNet50(weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 3s 0us/step


In [12]:
base_model.trainable= False

In [13]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(128,activation='relu')(x)
x=Dropout(0.5)(x)
predictions=Dense(15,activation='softmax')(x)

In [14]:
model=tf.keras.Model(inputs=base_model.input,outputs=predictions)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [15]:
history = model.fit(train_dataset, epochs=2, validation_data=test_dataset)
loss, accuracy = model.evaluate(test_dataset)

Epoch 1/2
57/57 [==============================] - 15s 153ms/step - loss: 0.1833 - accuracy: 0.9461 - val_loss: 0.0509 - val_accuracy: 0.9950
Epoch 2/2
7/7 [==============================] - 1s 89ms/step - loss: 0.0501 - accuracy: 0.9950


In [16]:
print('Test Loss:', loss)

Test Loss: 0.050077036023139954


In [17]:
print('Test accuracy:', accuracy)

Test accuracy: 0.9950000047683716


In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [21]:
!pip install -q -U keras-tuner
import tensorflow as tf
import keras_tuner
from kerastuner.tuners import RandomSearch

In [22]:
train_dir = '/content/train'
test_dir = '/content/validation'

In [23]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224,224),
    batch_size=32,
    label_mode='int'
)

Found 1800 files belonging to 2 classes.


In [24]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224,224),
    batch_size=32,
    label_mode='int'
)

Found 200 files belonging to 2 classes.


In [25]:
num_classes= 2

In [ ]:
def create_model(hp):
  model=tf.keras.Sequnetial([
      tf.keras.layers.COnv2D(32,kernel_size=(3,3),.)
  ])